In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv(r'C:\Users\pc\Desktop\Phone addiction challenge\train.csv')
test = pd.read_csv(r'C:\Users\pc\Desktop\Phone addiction challenge\test.csv')

In [3]:
train.head()

,id,Name,Age,Gender,Location,School_Grade,Daily_Usage_Hours,Sleep_Hours,Academic_Performance,Social_Interactions,...,Screen_Time_Before_Bed,Phone_Checks_Per_Day,Apps_Used_Daily,Time_on_Social_Media,Time_on_Gaming,Time_on_Education,Phone_Usage_Purpose,Family_Communication,Weekend_Usage_Hours,Addiction_Level
0,0,Safia Daoudi,18,Prefer not to say,Médéa,12th,5.1,8.5,85,10,...,0.5,122,15,3.2,1.0,0.2,Education,8,8.7,10.0
1,1,Soufiane Merabet,18,Female,Ouled Naïl,8th,6.2,8.2,86,4,...,1.1,141,5,4.6,2.4,0.0,Education,1,6.6,10.0
2,2,Yacine Bahri,16,Female,Chabet el Ameur,8th,4.1,6.0,80,6,...,1.0,149,14,1.8,1.9,1.2,Browsing,1,5.8,10.0
3,3,Hayat Bachiri,18,Male,Aïn Arnat,11th,4.3,8.0,74,9,...,1.3,22,16,2.6,2.4,1.9,Browsing,1,5.1,8.9
4,4,Fatima Lounes,17,Female,Chellata,11th,4.9,7.4,91,6,...,1.0,112,18,3.3,1.6,1.3,Social Media,3,9.0,10.0


In [4]:
test.head()

,id,Name,Age,Gender,Location,School_Grade,Daily_Usage_Hours,Sleep_Hours,Academic_Performance,Social_Interactions,...,Parental_Control,Screen_Time_Before_Bed,Phone_Checks_Per_Day,Apps_Used_Daily,Time_on_Social_Media,Time_on_Gaming,Time_on_Education,Phone_Usage_Purpose,Family_Communication,Weekend_Usage_Hours
0,0,Najia Louanchi,18,Female,Zerdaza,11th,5.3,7.5,97,9,...,0,0.6,109,5,1.8,1.1,1.4,Browsing,10,8.5
1,1,Oumaima Khelifi,19,Female,Tamanrasset,8th,3.6,7.8,59,9,...,1,1.7,85,16,4.0,1.2,0.7,Other,5,8.1
2,2,Chafik Adjal,14,Prefer not to say,El Oued,8th,2.4,6.0,84,3,...,1,0.3,65,20,2.1,1.5,1.4,Gaming,8,4.0
3,3,Leila Djaballah,15,Female,Khraicia,8th,4.3,6.7,93,8,...,0,1.9,74,12,3.1,0.2,0.5,Browsing,3,2.9
4,4,Wassila Sifi,17,Prefer not to say,Souk El Ténine,10th,5.0,7.2,80,8,...,1,1.7,122,16,2.0,2.2,1.2,Social Media,7,7.4


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2401 entries, 0 to 2400
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2401 non-null   int64  
 1   Name                    2401 non-null   object 
 2   Age                     2401 non-null   int64  
 3   Gender                  2401 non-null   object 
 4   Location                2401 non-null   object 
 5   School_Grade            2401 non-null   object 
 6   Daily_Usage_Hours       2401 non-null   float64
 7   Sleep_Hours             2401 non-null   float64
 8   Academic_Performance    2401 non-null   int64  
 9   Social_Interactions     2401 non-null   int64  
 10  Exercise_Hours          2401 non-null   float64
 11  Anxiety_Level           2401 non-null   int64  
 12  Depression_Level        2401 non-null   int64  
 13  Self_Esteem             2401 non-null   int64  
 14  Parental_Control        2401 non-null   

In [6]:
# Drop unnecessary columns
drop_cols = ['id', 'Name','Location']
train = train.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)

In [7]:
train.head()

,Age,Gender,School_Grade,Daily_Usage_Hours,Sleep_Hours,Academic_Performance,Social_Interactions,Exercise_Hours,Anxiety_Level,Depression_Level,...,Screen_Time_Before_Bed,Phone_Checks_Per_Day,Apps_Used_Daily,Time_on_Social_Media,Time_on_Gaming,Time_on_Education,Phone_Usage_Purpose,Family_Communication,Weekend_Usage_Hours,Addiction_Level
0,18,Prefer not to say,12th,5.1,8.5,85,10,1.4,8,2,...,0.5,122,15,3.2,1.0,0.2,Education,8,8.7,10.0
1,18,Female,8th,6.2,8.2,86,4,0.5,1,5,...,1.1,141,5,4.6,2.4,0.0,Education,1,6.6,10.0
2,16,Female,8th,4.1,6.0,80,6,1.3,9,2,...,1.0,149,14,1.8,1.9,1.2,Browsing,1,5.8,10.0
3,18,Male,11th,4.3,8.0,74,9,1.4,8,3,...,1.3,22,16,2.6,2.4,1.9,Browsing,1,5.1,8.9
4,17,Female,11th,4.9,7.4,91,6,0.9,4,1,...,1.0,112,18,3.3,1.6,1.3,Social Media,3,9.0,10.0


In [8]:
# Map School Grade (convert '12th' to 12)
def grade_to_num(grade):
    return int(grade.replace('th', '').replace('st', '').replace('nd', '').replace('rd', ''))

train['School_Grade'] = train['School_Grade'].apply(grade_to_num)
test['School_Grade'] = test['School_Grade'].apply(grade_to_num)

In [9]:
train.head()

,Age,Gender,School_Grade,Daily_Usage_Hours,Sleep_Hours,Academic_Performance,Social_Interactions,Exercise_Hours,Anxiety_Level,Depression_Level,...,Screen_Time_Before_Bed,Phone_Checks_Per_Day,Apps_Used_Daily,Time_on_Social_Media,Time_on_Gaming,Time_on_Education,Phone_Usage_Purpose,Family_Communication,Weekend_Usage_Hours,Addiction_Level
0,18,Prefer not to say,12,5.1,8.5,85,10,1.4,8,2,...,0.5,122,15,3.2,1.0,0.2,Education,8,8.7,10.0
1,18,Female,8,6.2,8.2,86,4,0.5,1,5,...,1.1,141,5,4.6,2.4,0.0,Education,1,6.6,10.0
2,16,Female,8,4.1,6.0,80,6,1.3,9,2,...,1.0,149,14,1.8,1.9,1.2,Browsing,1,5.8,10.0
3,18,Male,11,4.3,8.0,74,9,1.4,8,3,...,1.3,22,16,2.6,2.4,1.9,Browsing,1,5.1,8.9
4,17,Female,11,4.9,7.4,91,6,0.9,4,1,...,1.0,112,18,3.3,1.6,1.3,Social Media,3,9.0,10.0


In [10]:
# Define target and features
target = 'Addiction_Level'
features = [col for col in train.columns if col != target]

In [11]:
# Define categorical columns (CatBoost handles them automatically)
categorical_features = ['Gender', 'Phone_Usage_Purpose']

In [12]:
# Split data
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [13]:
# Create CatBoost Pool 
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

In [14]:
import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
import numpy as np

# Objective function for Optuna
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1500),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_strength": trial.suggest_float("random_strength", 0, 1),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "RMSE",
        "cat_features": categorical_features,
        "verbose": 0,
        "random_seed": 42
    }

    model = CatBoostRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error")
    return -np.mean(score)

# Run the study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)

# Save best parameters
best_params = study.best_params


c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-23 19:16:19,379] A new study created in memory with name: no-name-da131574-9315-47ac-91cc-a506bdd8fcf2
[I 2025-07-23 19:17:14,033] Trial 0 finished with value: 0.18668487900305938 and parameters: {'iterations': 1024, 'depth': 4, 'learning_rate': 0.009909797052500526, 'l2_leaf_reg': 8.830030879425781, 'bagging_temperature': 0.2102473664540434, 'random_strength': 0.4276800809095168, 'border_count': 143}. Best is trial 0 with value: 0.18668487900305938.
[I 2025-07-23 19:19:54,944] Trial 1 finished with value: 0.22699435613764088 and parameters: {'iterations': 1112, 'depth': 10, 'learning_rate': 0.29125627779710506, 'l2_leaf_reg': 6.231579090918251, 'bagging_temperature': 0.36336991892189385, 'random_strength

In [15]:
# Train CatBoost Regressor with best hyperparameters
best_params["loss_function"] = "RMSE"
best_params["cat_features"] = categorical_features
best_params["random_seed"] = 42
best_params["verbose"] = 100

model = CatBoostRegressor(**best_params)
model.fit(train_pool)

0:	learn: 1.5799188	total: 19.6ms	remaining: 20.1s
100:	learn: 0.3803809	total: 2.17s	remaining: 19.9s
200:	learn: 0.2530624	total: 4.3s	remaining: 17.6s
300:	learn: 0.2155933	total: 6.4s	remaining: 15.4s
400:	learn: 0.1924706	total: 8.51s	remaining: 13.2s
500:	learn: 0.1756832	total: 10.6s	remaining: 11.1s
600:	learn: 0.1610423	total: 12.7s	remaining: 8.98s
700:	learn: 0.1501914	total: 14.8s	remaining: 6.85s
800:	learn: 0.1408239	total: 16.9s	remaining: 4.73s
900:	learn: 0.1321758	total: 19s	remaining: 2.61s
1000:	learn: 0.1240580	total: 21.1s	remaining: 505ms
1024:	learn: 0.1224217	total: 21.6s	remaining: 0us


In [16]:
# Predict on training set
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f"Training MSE: {mse_train:.4f}")

Training MSE: 0.0156


In [17]:
# Predict on test set
y_pred_test = model.predict(X_test)

In [18]:
# Save predictions
submission = pd.DataFrame({
    'id': pd.read_csv(r'C:\Users\pc\Desktop\Phone addiction challenge\test.csv')['id'],
    'Addiction_Level_Prediction': y_pred_test
})
submission.to_csv(r'C:\Users\pc\Desktop\Phone addiction challenge\phone_addiction_predictions_ctb-HP.csv', index=False)
print("CatBoost predictions saved!")

CatBoost predictions saved!
